<a href="https://colab.research.google.com/github/youssef-ASSOIL/OpenFoodFacts_spark/blob/main/OpenFoodFacts_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>